In [1]:

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d

# import ipywidgets.interact class, this class will represent a slide bar.
from ipywidgets import interact
import math
from numpy import cos as cos
from numpy import sin as sin

In [2]:
# use interact decorator to decorate the function, so the function can receive the slide bar's value with parameter x.
# add another slide bar to this function with the interact decorator.   
# b1 = base rotation, j1 = joint 1 ,j2 = joint 2, j3 = joint 3 ^ wrist 1
'''

l1,l2,l3 = 10,10,10
def realqs(x,y,angle):
    q_z = angle
    x2 = x - (l3 * np.cos(q_z))
    y2 = y - (l3 * np.sin(q_z))
    q2 = (x2**2 + y2**2 - l1**2 - l2**2) / (2 * l1 * l2)
    q1 = -np.arctan( (l2* np.cos(q2)) / (l1+(l2 * np.sin(q2)))) + np.arctan(y2/x2)
    return q1,q2,q_z
    
q1,q2,q3 = zip(getq(0.1,15,55))
q1,q2,q3 = q1[0],q2[0],q3[0]



def mapping_function(b1,j1,j2,j3):
    x = l1 * np.cos(q1) +  l2 * np.cos(q1 + q2) + l3 * np.cos(q1 + q2 + q3)
    y = l1 * np.sin(q1) +  l2 * np.sin(q1 + q2) + l3 * np.sin(q1 + q2 + q3)
    
    print(x, y)
mapping_function(0,q1,q2,q3)
'''



'\n\nl1,l2,l3 = 10,10,10\ndef realqs(x,y,angle):\n    q_z = angle\n    x2 = x - (l3 * np.cos(q_z))\n    y2 = y - (l3 * np.sin(q_z))\n    q2 = (x2**2 + y2**2 - l1**2 - l2**2) / (2 * l1 * l2)\n    q1 = -np.arctan( (l2* np.cos(q2)) / (l1+(l2 * np.sin(q2)))) + np.arctan(y2/x2)\n    return q1,q2,q_z\n    \nq1,q2,q3 = zip(getq(0.1,15,55))\nq1,q2,q3 = q1[0],q2[0],q3[0]\n\n\n\ndef mapping_function(b1,j1,j2,j3):\n    x = l1 * np.cos(q1) +  l2 * np.cos(q1 + q2) + l3 * np.cos(q1 + q2 + q3)\n    y = l1 * np.sin(q1) +  l2 * np.sin(q1 + q2) + l3 * np.sin(q1 + q2 + q3)\n    \n    print(x, y)\nmapping_function(0,q1,q2,q3)\n'

In [3]:
l1,l2,l3 = 30,30,10
def getq(x,y,gripperangle):
    print(x,y)
    gripperangle = gripperangle * (np.pi / 180)
    p0 = np.matrix([[x - l3 * np.cos(gripperangle)],
                  [y - l3 * np.sin(gripperangle)]])

#     q2 = np.pi - np.arccos((l1*l1 + l2*l2 - (p0.T * p0)) / (2 * l1 * l2))
    q2 = np.pi - np.arccos((l1**2 + l2**2 - ( (x - l3 * np.cos(gripperangle))**2 + (y - l3 * np.sin(gripperangle))**2 )) / (2 * l1 * l2))
    q2_2 = -np.arccos( ( (p0.T * p0)  - l1*l1 - l2*l2 ) / (2 * l1 * l2))
    q1 = np.arctan( (y - l3 * np.sin(gripperangle)) / (x - l3 * np.cos(gripperangle)) ) - np.arctan( (l2 * np.sin(q2)) / (l1 + l2 * np.cos(q2)))
    
    q1_2 = np.arctan( (y - l3 * np.sin(gripperangle)) / (x - l3 * np.cos(gripperangle)) ) - np.arctan( (l2 * np.sin(q2_2)) / (l1 + l2 * np.cos(q2_2)))
    
#     q1_2 = q1
#     q2_2 = q2
    q3 = gripperangle - (q1 + q2)
    return np.float(q1), float(q2), float(q3),float(q2_2), float(q1_2)

In [4]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (10,7)
@interact(x=(0,40),y=(-20,40),gripperangle=(-90,90))
def hmpf(x,y,gripperangle):
    fig = plt.figure()
    ax = plt.axes()
    
    q1,q2,q3,q2_2,q1_2 = zip(getq(x,y,gripperangle))
    q1,q2,q3,q2_2,q1_2 = q1[0],q2[0] ,q3[0], q2_2[0], q1_2[0] 
    print('Q angles')
    print(q1 * (180 / np.pi),q2 * (180 / np.pi),q3 * (180 / np.pi), q1_2 * (180 / np.pi),q2_2 * (180 / np.pi))
    
    servo1 = q1 
    servo2 = q2
    servo3 = q3 
    servo2_2 = q2_2
    servo1_2 = q1_2
    
    xm = l1 * np.cos(q1) +  l2 * np.cos(q1 + q2) + l3 * np.cos(q1 + q2 + q3)
    ym = l1 * np.sin(q1) +  l2 * np.sin(q1 + q2) + l3 * np.sin(q1 + q2 + q3)
    arm_pos_vector = np.array([[xm,ym]]).T
    
    ax.quiver(0,0,x,y, scale=1, angles='xy', scale_units='xy', color='red',alpha=0.8)
    ax.quiver(0,0,xm,ym, scale=1, angles='xy', scale_units='xy', color='blue', alpha=0.8)
    
#     A1
    
    A1_vector = np.matrix([[np.cos(servo1), -np.sin(servo1)],[np.sin(servo1), np.cos(servo1)]]) * np.matrix([[l1,0]]).T
    a1x = A1_vector[0,0]
    a1y = A1_vector[1,0]
    ax.quiver(0,0,a1x, a1y, scale=1, angles='xy', scale_units='xy', color='green', alpha=0.8)

#     A2
    A2_vector = (np.matrix([[np.cos(servo1 + servo2), -np.sin(servo1 + servo2)],
                             [np.sin(servo1 +servo2), np.cos(servo1 + servo2)]]) * np.matrix([[l2,0]]).T)
    a2x = A2_vector[0,0]
    a2y = A2_vector[1,0]
    ax.quiver(a1x,a1y,a2x, a2y, scale=1, angles='xy', scale_units='xy', color='green', alpha=0.8)
    '''
    
#     EXPERIMENTAL
#     A1.2
    angle_arm_pos = float(np.arccos( (arm_pos_vector.T * A1_vector) /
                              ((np.linalg.norm(arm_pos_vector)) * (np.linalg.norm(A1_vector))) ))
    print(f'ANLGE{angle_arm_pos * (180/np.pi)}')
    print(angle_arm_pos)
    
    A1_2Vector = (np.matrix([[np.cos(2*angle_arm_pos), -np.sin(2*angle_arm_pos)],
                             [np.sin(2*angle_arm_pos), np.cos(2*angle_arm_pos)]]) * A1_vector)
    print(A1_2Vector)
    a12x = A1_2Vector[0,0]
    a12y = A1_2Vector[1,0]
    ax.quiver(0,0,a12x, a12y, scale=1, angles='xy', scale_units='xy', color='yellow', alpha=0.8)
                              
    
    #     A2.2
#     A2_2Vector = np.matrix([[np.cos( servo1_2 + servo2_2), -np.sin(servo1_2 + servo2_2)],
#                              [np.sin(servo1_2 + servo2_2), np.cos(servo1_2 + servo2_2)]]) * 
#                   np.matrix([[l2,0]]).T
#     a22x = A2_2Vector[0,0]
#     a22y = A2_2Vector[1,0]
    ax.quiver(a12x,a12y, (a1x + a2x) -a12x ,(a1y + a2y) - a12y, scale=1, angles='xy', scale_units='xy', color='yellow', alpha=0.8)
    
    '''
#     A1
    
    A1_vector = np.matrix([[np.cos(servo1_2), -np.sin(servo1_2)],[np.sin(servo1_2), np.cos(servo1_2)]]) * np.matrix([[l1,0]]).T
    a1x = A1_vector[0,0]
    a1y = A1_vector[1,0]
    ax.quiver(0,0,a1x, a1y, scale=1, angles='xy', scale_units='xy', color='yellow', alpha=0.8)

#     A2
    A2_vector = (np.matrix([[np.cos(servo1_2 + servo2_2), -np.sin(servo1_2 + servo2_2)],
                             [np.sin(servo1_2 +servo2_2), np.cos(servo1_2 + servo2_2)]]) * np.matrix([[l2,0]]).T)
    a2x = A2_vector[0,0]
    a2y = A2_vector[1,0]
    ax.quiver(a1x,a1y,a2x, a2y, scale=1, angles='xy', scale_units='xy', color='yellow', alpha=0.8)
    
    
#     A3
    a3x,a3y = zip(np.matrix([[np.cos(servo1 + servo2 + servo3), -np.sin(servo1 + servo2+servo3)],
                             [np.sin(servo1 +servo2+servo3), np.cos(servo1 + servo2+servo3)]]) *
                  np.matrix([[l3,0]]).T)
    a3x = a3x[0]
    a3y = a3y[0]
    ax.quiver(a1x + a2x,a1y + a2y,a3x, a3y, scale=1, angles='xy', scale_units='xy', color='green', alpha=0.8)
    
    ax.set_xlabel('X')
    ax.set_xlim(-10,40)
    ax.set_ylim(-10,40)
    plt.show()

interactive(children=(IntSlider(value=20, description='x', max=40), IntSlider(value=10, description='y', max=4…

In [21]:
l1,l2,l3 = 30,30,10
def getq_3D(x,y,z,gripperangle):
    
    base_angle = np.arctan(y/x) # Bereken de rotatie om de Z as, ookwel de base angle
    q0 = base_angle
    print(q0)
    input_x = np.matrix([[x],
                         [y],
                         [z]])
    #Rotatie van de input coordinaten vector met de minus base angle, dit resulteert in een vector in die 
    #geen Y component meer heeft, hij bestaat nu alleen in de X-Z plane, 
    #nu kunnen we met deze output vector de oude 2D inverse kinematics toepassen en krijgen we de juiste angles
    #voor de arm
    
    output_x = np.matrix([[np.cos(-base_angle), -np.sin(-base_angle), 0],
                        [np.sin(-base_angle), np.cos(-base_angle),0],[0,0,1]]) * input_x

#     2D portion (x, z)-------------------------
    
    gripperangle = gripperangle * (np.pi / 180)
    p0 = np.matrix([[output_x[0,0] - l3 * np.cos(gripperangle)],
                  [z - l3 * np.sin(gripperangle)]])
#Q2 heeft meerdere oplossingen
#     q2 = np.pi - np.arccos((l1*l1 + l2*l2 - (p0.T * p0)) / (2 * l1 * l2))
#     q2 = np.pi - np.arccos((l1**2 + l2**2 - ( (output_x[0] - l3 * np.cos(gripperangle))**2 + (y - l3 * np.sin(gripperangle))**2 )) / (2 * l1 * l2))
    q2_2 = -np.arccos( ( (p0.T * p0)  - l1*l1 - l2*l2 ) / (2 * l1 * l2))
    print(f'Q2 test: { ( (p0.T * p0)  - l1*l1 - l2*l2 ) / (2 * l1 * l2)}')
#     q1 = np.arctan( (y - l3 * np.sin(gripperangle)) / (output_x[0] - l3 * np.cos(gripperangle)) ) - np.arctan( (l2 * np.sin(q2)) / (l1 + l2 * np.cos(q2)))
    
    q1_2 = np.arctan( (z - l3 * np.sin(gripperangle)) / (output_x[0,0] - l3 * np.cos(gripperangle)) ) - np.arctan( (l2 * np.sin(q2_2)) / (l1 + l2 * np.cos(q2_2)))
    
#     q1_2 = q1
#     q2_2 = q2
    q3 = gripperangle - (q1_2 + q2_2)
    return np.float(q0), float(q1_2), float(q2_2),float(q3), output_x

In [22]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (10,7)
@interact(x=(0,40),y=(0,40),z=(0,40),gripperangle=(-90,90))
def hmpf(x,y,z,gripperangle):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    
    q0,q1,q2,q3,output_x = zip(getq_3D(x,y,z,gripperangle))
    
    q0= q0[0]
    q1= q1[0]
    q2= q2[0]
    q3= q3[0]
    print(f'ANGLES {q0 * (180 / np.pi), q1 * (180 / np.pi), q2 * (180 / np.pi), q3 * (180 / np.pi)}')
    
       
    ax.quiver(0,0,0, x,y,z,  color='red', alpha=0.8)
    #PROJECTION ONTO X-Z PLANE  
    ax.quiver(0,0,0, output_x[0][0],output_x[0][1],output_x[0][2],  color='darkred', alpha=0.8)
    
    
    base_rotation_matrix = np.matrix([[np.cos(q0), -np.sin(q0),0], 
                                     [np.sin(q0),np.cos(q0), 0],
                                     [0,0,1]])
    #A1
    A1_vector = base_rotation_matrix * np.matrix([[np.cos(-q1), 0, np.sin(-q1)], 
                           [0,1,0], 
                           [-np.sin(-q1),0, np.cos(-q1)] ]) * np.matrix([[l1],[0],[0]])
    
    ax.quiver(0,0,0, A1_vector[0],A1_vector[1],A1_vector[2],  color='green', alpha=0.8)
    
     #A2
    A2_vector = base_rotation_matrix *  np.matrix([[np.cos( -(q1 + q2)), 0, np.sin(-(q1 + q2))], 
                           [0,1,0], 
                           [-np.sin(-(q1 + q2)),0, np.cos(-(q1 + q2))] ]) * np.matrix([[l2],[0],[0]])
    
    ax.quiver(A1_vector[0],A1_vector[1],A1_vector[2], A2_vector[0],A2_vector[1],A2_vector[2],  color='green', alpha=0.8)
    
    #A3
    A3_vector = base_rotation_matrix * np.matrix([[np.cos( -(q1 + q2 + q3)), 0, np.sin(-(q1 + q2 + q3))], 
                           [0,1,0], 
                           [-np.sin(-(q1 + q2 + q3)),0, np.cos(-(q1 + q2 + q3))] ]) * np.matrix([[l3],[0],[0]])
    ax.quiver(A1_vector[0] + A2_vector[0],A1_vector[1] + A2_vector[1],A1_vector[2] + A2_vector[2], 
              A3_vector[0],A3_vector[1],A3_vector[2],  color='green', alpha=0.8)
    
   
    '''
    #     Final position of the robot arm
    xm = (l1 * np.cos(q1) +  l2 * np.cos(q1 + q2) + l3 * np.cos(q1 + q2 + q3))
    ym = 0
    zm = l1 * np.sin(q1) +  l2 * np.sin(q1 + q2) + l3 * np.sin(q1 + q2 + q3)
    arm_output = np.matrix([[np.cos(-q0), -np.sin(-q0), 0],
                        [np.sin(-q0), np.cos(-q0),0],
                            [0,0,1]]) * np.matrix([[xm],[ym], [zm]])
    
    ax.quiver(0,0,0,arm_output[0],arm_output[1],arm_output[2],  color='yellow', alpha=0.8)
    '''

    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')           
    ax.set_xlim(0,40)
    ax.set_ylim(0,40)
    ax.set_zlim(0,40)
    plt.show()

interactive(children=(IntSlider(value=20, description='x', max=40), IntSlider(value=20, description='y', max=4…

In [ ]:
    Serial.print("MATRIX P0: ");